<a href="https://colab.research.google.com/github/ShaunakSen/Contextual-hyperlinks/blob/master/Dataset_extraction_%2B_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/google-research-datasets/wiki-links/raw/master/data-00001.gz

--2019-08-18 13:16:32--  https://github.com/google-research-datasets/wiki-links/raw/master/data-00001.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research-datasets/wiki-links/master/data-00001.gz [following]
--2019-08-18 13:16:33--  https://raw.githubusercontent.com/google-research-datasets/wiki-links/master/data-00001.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17111634 (16M) [application/octet-stream]
Saving to: ‘data-00001.gz’

data-00001.gz       100%[===================>]  16.32M   101MB/s    in 0.2s    

2019-08-18 13:16:34 (101 MB/s) - ‘data-00001.gz’ saved [17111634/17111

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import urllib.request
import os
import zipfile
import re

import requests
from bs4 import BeautifulSoup

import nltk
import string

import pickle
from urllib.parse import urlparse

In [0]:
test_url = "http://1158pm.info/food.html"

parsed_url = urlparse(test_url)

print (parsed_url.netloc)

1158pm.info


In [0]:
!python -m nltk.downloader all

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipp

In [0]:
!gzip -d /content/data-00001.gz

In [0]:
from google.colab import files

uploaded = files.upload()

Saving data-00001 to data-00001


In [0]:
f = open('./gdrive/My Drive/data-00001', 'r')

lines = f.readlines()


In [0]:
def process_url(url):
  """
  function to determine if to keep a URL or not
  """
  # check start of url - http or https
  if not url.startswith("http"):
    return False
  # check first characters and extensions
  if re.match(pattern='(http|https)://\w.*', string=url) is None or url.endswith("pdf") or url.endswith("doc") or url.endswith("docx"):
    return False
  # all passed return True
  return True

total_urls = 0 # etsimate of total source urls in the dataset: may contain duplicates 

for line in lines:
  if (line.startswith("URL")):
    url = line.split()[1]
    # check if URL valid
    if process_url(url):
      total_urls+=1


print ("Total URLs:", total_urls)

Total URLs: 99494


### Build Dataset

Dataset format:

```
[[mention_text, source, target], ...]
```

In [0]:
# [[mention_text, source, target], ...]

dataset = []

for i in range(len(lines)):
  if (lines[i].startswith("URL")):
    url = lines[i].split()[1]
    # check if URL valid
    if process_url(url):
      # look at the MENTION tags
      curr_idx = i+1
      next_line = lines[curr_idx]
      while next_line.startswith("MENTION"):
        # MENTION detected
        # split by ' '
        mention_text = next_line.split()
        # get mention tokens
        mention_token = " ".join(mention_text[1:len(mention_text)-2])
        # get target link
        target = mention_text[len(mention_text)-1]
        # append in form: [mention_text, source, target]
        dataset.append([mention_token, url, target])
        curr_idx += 1
        next_line = lines[curr_idx]


In [0]:
dataset[:10]

[['Los Angeles Philharmonic',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Los_Angeles_Philharmonic'],
 ['Los Angeles Master Chorale',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Los_Angeles_Master_Chorale'],
 ['Walt Disney',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Walt_Disney'],
 ['Evo',
  'http://0-one.org/2012/07/14/ouya-99-android-game-console/',
  'http://en.wikipedia.org/wiki/EVO_Smart_Console'],
 ['Groupthink',
  'http://0.yogins.com/2012/01/30/groupthink-and-centralized-bathrooms-with-knowledge-spillovers/',
  'http://en.wikipedia.org/wiki/Groupthink'],
 ['Knowledge Spillovers',
  'http://0.yogins.com/2012/01/30/groupthink-and-centralized-bathrooms-with-knowledge-spillovers/',
  'http://en.wikipedia.org/wiki/Knowledge_spillover'],
 ['Latin',
  'http://00000000.ca/',
  'http://en.wikipedia.org/wiki/Lat

In [0]:
print (len(dataset))

373961


In [0]:
print ("MENTIONs per link:", len(dataset)/total_urls)

MENTIONs per link: 3.7586286610247854


In [0]:
# check how many have same source and target links

"""
source_target_pairs = []

for x in range(len(dataset)):
  if [dataset[x][1], dataset[x][2]] not in source_target_pairs:
    source_target_pairs.append([dataset[x][1], dataset[x][2]])

print (len(source_target_pairs))
"""

347867


In [0]:
347867/373961

0.9302226702784515

Over 93% of source-target pairs are unique. So we do not have to fiilter through the list to remove duplicates

We have now created a dataset for the file as:

```
[[MENTION_TOKEN, SOURCE, TARGET], ...]
```

Next step is scraping for information about SOURCE and TARGET and building a new dataset of form:

```
[SOURCE_TEXT...MENTION_TOKEN,TARGET_TEXT...],
<special_tokens>
[SOURCE_TEXT...MENTION_TOKEN,TARGET_TEXT...],
...
```


In [0]:
def textProcessingPipeline(text):
  """
  preprocess text to get a window of words around the <mention>
  1. split into words
  2. convert to lowercase
  3. remove punctuation
  4. remove stopwords
  """
  tokens = nltk.tokenize.word_tokenize(text)
  tokens = [w.lower() for w in tokens]
  # maketrans: If three arguments are passed, each character in the third argument is mapped to None
  table = str.maketrans('','',string.punctuation)
  punc_removed = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  punc_removed = [word for word in punc_removed if word.isalpha()]

  stopwords = set(nltk.corpus.stopwords.words('english'))

  stopwords_removed = [word for word in punc_removed if word not in stopwords]

  return stopwords_removed

In [0]:
mention_tokens = [data[0] for data in dataset]
source_links = [data[1] for data in dataset]
target_links = [data[2] for data in dataset]

print (len(mention_tokens)==len(source_links) == len(target_links) ==len(dataset))


True


In [0]:
print (mention_tokens[93])

Chechen people


In [0]:
idx_to_check = 9838

r  = requests.get(source_links[idx_to_check])
data = r.text
soup = BeautifulSoup(data)

matched_links = soup.find_all(name='a', attrs={'href': target_links[idx_to_check]})
matched_link = None


for link in matched_links:
  if link.string.strip() == mention_tokens[idx_to_check]:
    matched_link = link

original_token = matched_link.string
special_token = 'littlemini'
matched_link.string = special_token
original_token = str(original_token)

parent = matched_link.parent

text_corpus = []
window_size = 50
curr_window = 0
token_found = False

for text in parent.stripped_strings:
  if token_found is True and curr_window >= window_size:
    break 
  processed_text = textProcessingPipeline(text)
  for word in processed_text:
    if word == special_token:
      token_found = True
      word = original_token
    text_corpus.append(word)
    if token_found:
      curr_window += 1


      
print (len(text_corpus))

AttributeError: ignored

In [0]:
r  = requests.get(target_links[idx_to_check])
data = r.text
soup = BeautifulSoup(data)

heading = str(soup.find_all(name='h1', attrs={'id':'firstHeading'})[0].string)

target_corpus = []
paras = soup.find_all('p')
curr_para_idx = 0
curr_window = 0
window_size = 50


while curr_window < window_size and curr_para_idx < len(paras):
  para_text = paras[curr_para_idx].stripped_strings
  for text in para_text:
      processed_text = textProcessingPipeline(text)
      for word in processed_text:
        if curr_window >= window_size:
          break
        target_corpus.append(word)
        curr_window += 1
  curr_para_idx += 1

print (len(target_corpus))

50


In [0]:
def build_corpus(dataset, source_window_size, target_window_size, lower_limit=0, limit=100):
  """
  [SOURCE_TEXT...MENTION_TOKEN,TARGET_TEXT...],
  <special_tokens>
  [SOURCE_TEXT...MENTION_TOKEN,TARGET_TEXT...],
  ...
  """

  if limit > len(dataset):
    limit=len(dataset)

  final_corpus = []

  mention_tokens = [data[0] for data in dataset]
  source_links = [data[1] for data in dataset]
  target_links = [data[2] for data in dataset]
  curr_error_url = None

  for idx in range(lower_limit, limit):
    try:
      # corpus for this idx
      idx_corpus = []
      # flags
      source_ok, traget_ok = False, False

      # check  if source_url is OK
      source_url = urlparse(source_links[idx]).netloc
      if source_url == curr_error_url:
        print ('Skip:', source_links[idx])
        continue
      # ------------FOR SOURCE DATA SCRAPING------------
      # establish connection + convert to soup
      
      r  = requests.get(source_links[idx])
      data = r.text
      soup = BeautifulSoup(data)

      # find the correct link
      matched_links = soup.find_all(name='a', attrs={'href': target_links[idx]})
      matched_link = None

      for link in matched_links:
        if link.string.strip() == mention_tokens[idx]:
          matched_link = link

      if matched_link is None:
        print ("Could not find link text in source link for idx:", idx)
        continue

      # change the link text to a unique code, while saving the original text
      original_token = matched_link.string
      special_token = 'littlemini'
      matched_link.string = special_token
      original_token = str(original_token)

      # get parent of matched link
      parent = matched_link.parent

      curr_window = 0

      token_found = False

      for text in parent.stripped_strings:
        if token_found is True and curr_window >= source_window_size:
          break 
        processed_text = textProcessingPipeline(text)
        for word in processed_text:
          if word == special_token:
            token_found = True
            word = original_token
          idx_corpus.append(word)
          if token_found:
            curr_window += 1

      # at this point the SOURCE TEXT as well as MENTION_TOKEN is stored in idx_corpus
      # SOURCE OK - SET FLAG
      if len(idx_corpus) > 0:
        source_ok = True
        source_len = len(idx_corpus)


      # ------------FOR TARGET DATA SCRAPING------------

      # establish connection + convert to soup
      r  = requests.get(target_links[idx])
      data = r.text
      soup = BeautifulSoup(data)

      # Get heading
      heading = str(soup.find_all(name='h1', attrs={'id':'firstHeading'})[0].string)

      if heading is not None:
        heading = textProcessingPipeline(heading)
      
      for word in heading:
        idx_corpus.append(word)


      # get all paras
      paras = soup.find_all('p')
      curr_para_idx = 0
      curr_window = 0

      # loop till we get the reqd words or till we run out of data
      while curr_window < target_window_size and curr_para_idx < len(paras):
        # get text in para
        para_text = paras[curr_para_idx].stripped_strings
        for text in para_text:
            processed_text = textProcessingPipeline(text)
            for word in processed_text:
              if curr_window >= target_window_size:
                break
              idx_corpus.append(word)
              curr_window += 1
        curr_para_idx += 1

      # at this point the TARGET TEXT stored in idx_corpus
      # TARGET OK - SET FLAG
      if len(idx_corpus) > source_len:
        target_ok = True

      if source_ok and target_ok:
        final_corpus.append(idx_corpus)
      else:
        print ('ERROR: S:', source_links[idx], 'T:', target_links[idx])

    except:
      # probably source_url is wrong
      curr_error_url = urlparse(source_links[idx]).netloc
      print ('Error for idx:', idx, 'Link:', source_links[idx], 'root:', curr_error_url)
      continue
    if idx%100==0:
      print ("Done", idx)

  return final_corpus

```
0 - 5000, 5000 - 10,0000, 10,000 - 15,0000
...
95000 - 100000
100000 - 105,000
```

In [0]:
batch_size = 1000

curr_lower_limit = 39000

curr_upper_limit = curr_lower_limit+batch_size

while curr_upper_limit < 50000:
  batch_corpus = build_corpus(dataset, 50, 50, curr_lower_limit, curr_upper_limit)

  filename = "corpus" + str(curr_lower_limit) + "_to_" + str(curr_upper_limit) + ".pickle"
  pickle_out = open("./gdrive/My Drive/"+filename, "wb")
  pickle.dump(batch_corpus, pickle_out)
  pickle_out.close()

  print ("Dumped in pickle", curr_lower_limit, "to", curr_upper_limit) 

  curr_lower_limit += batch_size
  curr_upper_limit += batch_size

In [0]:
data = open('./gdrive/My Drive/corpus5000_to_10000.pickle', 'rb')

data = pickle.load(data)

print (len(data))

1995


In [0]:
print (len(dataset)/5000)

74.7922


In [0]:
print (len(final_corpus))

2133


In [0]:
final_corpus[1]

In [0]:
for corpus_data in final_corpus:
  final_corpus_2.append(corpus_data)

In [0]:
print (len(final_corpus_2))

8120
